In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton_krylov
from time import clock as clock
import scipy.sparse.linalg as spla

%load_ext Cython

In [ ]:
# residual function
def residual_nu(  sol1,
                solN,
                g,
                Dx,
                D,
                I,
                Dt,
                M ):
    """
    Input arrays are defined as memoryviews with prefix const which declares read only buffers
    
    """
    
    # define a memview on the residual vector
    res = np.empty(I, dtype = np.float64)
    
    # anode boundary condition
    res[0] = (solN[0] - sol1[0] 
            + M * Dt * (-D[1] * ( solN[1] - solN[0]) / ((Dx[0] + Dx[1]) / 2) ) / Dx[0])
    
    for i in range( 1, I-1 ):
        
        #
        # Godunov scheme - upwinding with (unlimited !) piecewise constant reconstruction
        #
        
        # calculate velocity - driving electric field
        velup = - 2.0 * (g[i+1] - g[i]) / (Dx[i+1] + Dx[i]) # velocity on right cell edge
        veldown = - 2.0 * (g[i] - g[i-1]) / (Dx[i] + Dx[i-1]) # velocity on left cell edge
        
        # switch between first order and second order with van leer limiter
        if (sol1[i] - sol1[i-1]) * (sol1[i+1] - sol1[i]) < 0:
            
            limiter = 0.0

        else:
            r = ( sol1[i] - sol1[i-1] ) / ( sol1[i+1] - sol1[i] )
            
            limiter = 2*r / ( 1.0 + r )
        
        if velup > 0:
            
            upflux = velup * D[i+1] * ( solN[i]
                     + Dx[i] * limiter * ( solN[i+1] - solN[i] ) / (Dx[i+1] + Dx[i]) )

        else:

            upflux = velup * D[i+1] * ( solN[i+1] 
                    - Dx[i+1]  * limiter * ( solN[i+1] - solN[i] ) / (Dx[i+1] + Dx[i]) )
        
        # flux at left cell edge
        if veldown > 0:
            
            downflux = veldown * D[i] * ( solN[i-1] 
                       + Dx[i-1] * limiter * (solN[i] - solN[i-1]) / (Dx[i] + Dx[i-1]) )

        else:
            
            
            downflux = veldown * D[i] * ( solN[i] 
                        - Dx[i] * limiter * solN[i] - solN[i-1]) / (Dx[i] + Dx[i-1]) )
        
        res[i] = ( solN[i] -sol1[i] 
        + M * Dt * ( -2.0 * D[i+1] * ( solN[i+1] -  solN[i]) / (Dx[i+1] + Dx[i]) 
                       + 2.0 * D[i]  * ( solN[i] -  solN[i-1]) / (Dx[i] + Dx[i-1]) 
                   upflux - downflux ) / Dx[i] )
    
        # velocity on f_left
    veldown = - 2.0 * (g[I-1] - g[I-2]) / (Dx[I-1] + Dx[I-2])
    velup = - 2.0 * (g[I-1] - g[I-2]) / (Dx[I-1] + Dx[I-2]) # periodic extrapolated other velocitiy
    
    if veldown > 0:

        downflux = veldown * D[I-1] * solN[I-2]

    else:

        downflux = veldown * D[I-1] * solN[I-1]
    
    if velup > 0:
            
            upflux = velup * D[I] * solN[I-1]

    else:

            upflux = velup * D[I] * solN[0]
    
    # catode boundary condition
    res[I-1] = (solN[I-1]- sol1[I-1] 
                + M * Dt * (D[I] * ( solN[I-1] - solN[I-2]) / ((Dx[I-1] + Dx[I-2]) / 2.0) ) / Dx[I-1])
    
    return res

def residual(sol_old, sol_new, g, Dx, D, I, Dt, M):
    """
    Implemented periodic boundary conditions
    """
    res = np.zeros(I, dtype = np.float64)
    
    #
    # Godunov scheme
    #
    
    veldown =  -2.0 * (g[1] - g[0]) / (Dx[1] + Dx[0]) # periodic
    velup = - 2.0 * (g[1] - g[0]) / (Dx[1] + Dx[0])  # velocity on right cell edge
    
    if veldown > 0:

            downflux = veldown * D[0] * sol_new[I-1]

    else:

            downflux = veldown * D[0] * sol_new[0]
            
    if velup >= 0:

        upflux = velup * D[1] * (sol_new[0]  )

    else:

        upflux = velup * D[1] * sol_new[1]
    
    # anode boundary condition
    res[0] = (sol_new[0] - sol_old[0] + M * Dt * (upflux - downflux) / Dx[0])

    # inner points
    for i in range(1,I-1):
        
        
          
        
        
        
            
        


            
    # catode boundary condition
    res[I-1] = (sol_new[I-1]- sol_old[I-1] + M * Dt * ( ) / Dx[I-1])
    
    return res